In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adadelta, Adagrad, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.utils import to_categorical

from six.moves import range
import numpy as np
import scipy as sp
from tensorflow.keras import backend as K  
import random
import scipy.io
import matplotlib.pyplot as plt


In [2]:
# Definition of the model

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

# Façon de definir un modèle en keras à la manière de pytorch


class Cnn_model(tf.keras.Model):
    def __init__(self):
        super().__init__('VGG8')

        self.conv1 = Conv2D(64, nb_conv, padding='same',
                            activation='relu')  # no regularization ?
        self.Pool1 = MaxPooling2D(nb_pool)
        self.drop1 = Dropout(0.25)

        self.conv2 = Conv2D(128, nb_conv, padding='same', activation='relu')
        self.Pool2 = MaxPooling2D(nb_pool)
        self.drop2 = Dropout(0.25)

        self.conv3 = Conv2D(256, nb_conv, padding='same', activation='relu')
        self.conv3bis = Conv2D(256, nb_conv, padding='same', activation='relu')
        self.Pool3 = MaxPooling2D(nb_pool)
        self.drop3 = Dropout(0.25)

        self.conv4 = Conv2D(512, nb_conv, padding='same', activation='relu')
        self.conv4bis = Conv2D(512, nb_conv, padding='same', activation='relu')
        self.Pool4 = MaxPooling2D(nb_pool)
        self.drop4 = Dropout(0.25)

        self.flatten = Flatten()
        self.dense = Dense(4096, activation='relu')
        self.drop_last = Dropout(0.5)
        self.classifier = Dense(nb_classes, activation='softmax')

    def call(self, x): # the forward method
        
        x = self.conv1(x)
        x = self.Pool1(x)
        x = self.drop1(x) 
        
        x = self.conv2(x)
        x = self.Pool2(x) 
        x = self.drop2(x)
        
        x = self.conv3(x)
        x = self.conv3bis(x)
        x = self.Pool3(x)
        x = self.drop3(x)
        
        x = self.conv4(x)
        x = self.conv4bis(x)
        x = self.Pool4(x)
        x = self.drop4(x)
        
        x = self.flatten(x)
        x = self.dense(x)
        x = self.drop_last(x)
        probas = self.classifier(x)
        
        return probas
    
    def predict_stochastic(self, x): # same same dorwar
        
        x = self.conv1(x)
        x = self.Pool1(x)
        x = self.drop1(x) 
        
        x = self.conv2(x)
        x = self.Pool2(x) 
        x = self.drop2(x)
        
        x = self.conv3(x)
        x = self.conv3bis(x)
        x = self.Pool3(x)
        x = self.drop3(x)
        
        x = self.conv4(x)
        x = self.conv4bis(x)
        x = self.Pool4(x)
        x = self.drop4(x)
        
        x = self.flatten(x)
        x = self.dense(x)
        x = self.drop_last(x)
        probas = self.classifier(x)
        
        return probas

In [3]:
class Vgg_like(tf.keras.Model):
    def __init__(self):
        super().__init__('VGG8')

        self.conv1 = Conv2D(32, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu')
        self.Pool1 = MaxPooling2D()
        self.drop1 = Dropout(0.25)
        
        self.conv2 = Conv2D(64, (3, 3), padding='same', activation='relu')
        self.flatten = Flatten()
        
        self.dense = Dense(128, activation='relu')
        self.drop_last = Dropout(0.5)
        self.classifier = Dense(nb_classes, activation='softmax')

    def call(self, x):
        
        x = self.conv1(x)
        x = self.Pool1(x)
        x = self.drop1(x) 
        
        x = self.conv2(x)
        
        x = self.flatten(x)
        x = self.dense(x)
        x = self.drop_last(x)
        probas = self.classifier(x)
        
        return probas
    
    def predict_stochastic(self, x):
        
        x = self.conv1(x)
        x = self.Pool1(x)
        x = self.drop1(x) 
        
        x = self.conv2(x)
        
        x = self.flatten(x)
        x = self.dense(x)
        x = self.drop_last(x)
        probas = self.classifier(x)
        
        return probas


In [ ]:
# from sklearn.metrics import log_loss
# from sklearn.metrics import mean_squared_error
# from math import sqrt


Experiments = 2

batch_size = 128
nb_classes = 10

# use a large number of epochs
nb_epoch = 10


score = 0
all_accuracy = 0
all_predicted_log_likelihood = 0
all_rmse = 0
acquisition_iterations = 90

# use a large number of dropout iterations
dropout_iterations = 10

Queries = 10


Experiments_All_Accuracy = np.zeros(shape=(acquisition_iterations+1))


for e in range(Experiments):

    print('Experiment Number ', e)

    # the data, shuffled and split between tran and test sets
    (X_train_All, y_train_All), (X_test, y_test) = mnist.load_data()

    X_train_All = X_train_All[:,:,:,None]
    X_test = X_test[:,:,:,None]
    
    
    random_split = np.asarray(random.sample(
        range(0, X_train_All.shape[0]), X_train_All.shape[0]))

    X_train_All = X_train_All[random_split, :, :, :]
    y_train_All = y_train_All[random_split]

    X_valid = X_train_All[10000:15000, :, :, :]
    y_valid = y_train_All[10000:15000]

    X_Pool = X_train_All[20000:60000, :, :, :]
    y_Pool = y_train_All[20000:60000]

    X_train_All = X_train_All[0:10000, :, :, :]
    y_train_All = y_train_All[0:10000]

    # training data to have equal distribution of classes
    idx_0 = np.array(np.where(y_train_All == 0)).T
    idx_0 = idx_0[0:10, 0]
    X_0 = X_train_All[idx_0, :, :, :]
    y_0 = y_train_All[idx_0]

    idx_1 = np.array(np.where(y_train_All == 1)).T
    idx_1 = idx_1[0:10, 0]
    X_1 = X_train_All[idx_1, :, :, :]
    y_1 = y_train_All[idx_1]

    idx_2 = np.array(np.where(y_train_All == 2)).T
    idx_2 = idx_2[0:10, 0]
    X_2 = X_train_All[idx_2, :, :, :]
    y_2 = y_train_All[idx_2]

    idx_3 = np.array(np.where(y_train_All == 3)).T
    idx_3 = idx_3[0:10, 0]
    X_3 = X_train_All[idx_3, :, :, :]
    y_3 = y_train_All[idx_3]

    idx_4 = np.array(np.where(y_train_All == 4)).T
    idx_4 = idx_4[0:10, 0]
    X_4 = X_train_All[idx_4, :, :, :]
    y_4 = y_train_All[idx_4]

    idx_5 = np.array(np.where(y_train_All == 5)).T
    idx_5 = idx_5[0:10, 0]
    X_5 = X_train_All[idx_5, :, :, :]
    y_5 = y_train_All[idx_5]

    idx_6 = np.array(np.where(y_train_All == 6)).T
    idx_6 = idx_6[0:10, 0]
    X_6 = X_train_All[idx_6, :, :, :]
    y_6 = y_train_All[idx_6]

    idx_7 = np.array(np.where(y_train_All == 7)).T
    idx_7 = idx_7[0:10, 0]
    X_7 = X_train_All[idx_7, :, :, :]
    y_7 = y_train_All[idx_7]

    idx_8 = np.array(np.where(y_train_All == 8)).T
    idx_8 = idx_8[0:10, 0]
    X_8 = X_train_All[idx_8, :, :, :]
    y_8 = y_train_All[idx_8]

    idx_9 = np.array(np.where(y_train_All == 9)).T
    idx_9 = idx_9[0:10, 0]
    X_9 = X_train_All[idx_9, :, :, :]
    y_9 = y_train_All[idx_9]

    X_train = np.concatenate(
        (X_0, X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9), axis=0)
    y_train = np.concatenate(
        (y_0, y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9), axis=0)

    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')

    print('Distribution of Training Classes:', np.bincount(y_train))

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_valid = X_valid.astype('float32')
    X_Pool = X_Pool.astype('float32')
    X_train /= 255
    X_valid /= 255
    X_Pool /= 255
    X_test /= 255

    Y_test = to_categorical(y_test, nb_classes)
    Y_valid = to_categorical(y_valid, nb_classes)
    Y_Pool = to_categorical(y_Pool, nb_classes)

    # loss values in each experiment
    Pool_Valid_Loss = np.zeros(shape=(nb_epoch, 1))
    Pool_Train_Loss = np.zeros(shape=(nb_epoch, 1))
    Pool_Valid_Acc = np.zeros(shape=(nb_epoch, 1))
    Pool_Train_Acc = np.zeros(shape=(nb_epoch, 1))
    x_pool_All = np.zeros(shape=(1))

    Y_train = to_categorical(y_train, nb_classes)

    print('Training Model Without Acquisitions in Experiment', e)

    model = Vgg_like()
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model(X_train[:1])
    model.summary()
    
    hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
                     verbose=1, validation_data=(X_valid, Y_valid))
    Train_Result_Optimizer = hist.history
    Train_Loss = np.asarray(Train_Result_Optimizer.get('loss'))
    Train_Loss = np.array([Train_Loss]).T
    Valid_Loss = np.asarray(Train_Result_Optimizer.get('val_loss'))
    Valid_Loss = np.asarray([Valid_Loss]).T
    Train_Acc = np.asarray(Train_Result_Optimizer.get('accuracy'))
    Train_Acc = np.array([Train_Acc]).T
    Valid_Acc = np.asarray(Train_Result_Optimizer.get('val_accuracy'))
    Valid_Acc = np.asarray([Valid_Acc]).T

    Pool_Train_Loss = Train_Loss
    Pool_Valid_Loss = Valid_Loss
    Pool_Train_Acc = Train_Acc
    Pool_Valid_Acc = Valid_Acc
    

    print('Evaluating Test Accuracy Without Acquisition')
    score, acc = model.evaluate(X_test, Y_test, verbose=1)
    # Y_predict_probabilities = model.predict_proba(X_test, batch_size=batch_size)
    # predicted_log_likelihood = log_loss(Y_test, Y_predict_probabilities)
    # rmse = sqrt(mean_squared_error(Y_test, Y_predict_probabilities))

    # all_predicted_log_likelihood = predicted_log_likelihood
    # all_accuracy = acc
    # all_rmse = rmse

    all_accuracy = acc

    print('Starting Active Learning in Experiment ', e)

    for i in range(acquisition_iterations):

        print('POOLING ITERATION', i)

        # take subset of Pool Points for Test Time Dropout
        # and do acquisition from there
        pool_subset = 2000
        pool_subset_dropout = np.asarray(random.sample(
            range(0, X_Pool.shape[0]), pool_subset))
        X_Pool_Dropout = X_Pool[pool_subset_dropout, :, :, :]
        y_Pool_Dropout = y_Pool[pool_subset_dropout]

        score_All = np.zeros(shape=(X_Pool_Dropout.shape[0], nb_classes))
        All_Entropy_Dropout = np.zeros(shape=X_Pool_Dropout.shape[0])

        for d in range(dropout_iterations):
            print('Dropout Iteration', d)
            dropout_score = model.predict_stochastic(
                X_Pool_Dropout)
            # np.save('/Users/Riashat/Documents/Cambridge_THESIS/Code/Experiments/keras/active_learning/Acquisition_Functions/Bayesian_Active_Learning/GPU/BALD/Dropout_Scores/'+ 'Experiment_' + str(e)  + '_Dropout_Score_'+str(d)+'.npy',dropout_score)
            # computing G_X
            score_All = score_All + dropout_score

            # computing F_X
            dropout_score_log = np.log2(dropout_score)
            Entropy_Compute = - np.multiply(dropout_score, dropout_score_log)
            Entropy_Per_Dropout = np.sum(Entropy_Compute, axis=1)

            All_Entropy_Dropout = All_Entropy_Dropout + Entropy_Per_Dropout

        Avg_Pi = np.divide(score_All, dropout_iterations)
        Log_Avg_Pi = np.log2(Avg_Pi)
        Entropy_Avg_Pi = - np.multiply(Avg_Pi, Log_Avg_Pi)
        Entropy_Average_Pi = np.sum(Entropy_Avg_Pi, axis=1)

        G_X = Entropy_Average_Pi

        Average_Entropy = np.divide(All_Entropy_Dropout, dropout_iterations)

        F_X = Average_Entropy

        U_X = G_X - F_X

        # THIS FINDS THE MINIMUM INDEX
        # a_1d = U_X.flatten()
        # x_pool_index = a_1d.argsort()[-Queries:]

        a_1d = U_X.flatten()
        x_pool_index = a_1d.argsort()[-Queries:][::-1]

        # store all the pooled images indexes
        x_pool_All = np.append(x_pool_All, x_pool_index)

        # saving pooled images

        # #save only 3 images per iteration
        # for im in range(x_pool_index[0:2].shape[0]):
        # 	Image = X_Pool[x_pool_index[im], :, :, :]
        # 	img = Image.reshape((28,28))
        #sp.misc.imsave('/home/ri258/Documents/Project/Active-Learning-Deep-Convolutional-Neural-Networks/ConvNets/Cluster_Experiments/Dropout_Bald/Pooled_Images/' + 'Experiment_' + str(e) + 'Pool_Iter'+str(i)+'_Image_'+str(im)+'.jpg', img)

        Pooled_X = X_Pool_Dropout[x_pool_index,]
        Pooled_Y = y_Pool_Dropout[x_pool_index]

        # first delete the random subset used for test time dropout from X_Pool
        # Delete the pooled point from this pool set (this random subset)
        # then add back the random pool subset with pooled points deleted back to the X_Pool set
        delete_Pool_X = np.delete(X_Pool, (pool_subset_dropout), axis=0)
        delete_Pool_Y = np.delete(y_Pool, (pool_subset_dropout), axis=0)

        delete_Pool_X_Dropout = np.delete(
            X_Pool_Dropout, (x_pool_index), axis=0)
        delete_Pool_Y_Dropout = np.delete(
            y_Pool_Dropout, (x_pool_index), axis=0)

        X_Pool = np.concatenate((X_Pool, X_Pool_Dropout), axis=0)
        y_Pool = np.concatenate((y_Pool, y_Pool_Dropout), axis=0)

        print('Acquised Points added to training set')

        X_train = np.concatenate((X_train, Pooled_X), axis=0)
        y_train = np.concatenate((y_train, Pooled_Y), axis=0)

        # convert class vectors to binary class matrices
        Y_train = to_categorical(y_train, nb_classes)

        
        model = Vgg_like()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
        model(X_train[:1])
        
        hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
                         verbose=1, validation_data=(X_valid, Y_valid))
        
        Train_Result_Optimizer = hist.history
        Train_Loss = np.asarray(Train_Result_Optimizer.get('loss'))
        Train_Loss = np.array([Train_Loss]).T
        Valid_Loss = np.asarray(Train_Result_Optimizer.get('val_loss'))
        Valid_Loss = np.asarray([Valid_Loss]).T
        Train_Acc = np.asarray(Train_Result_Optimizer.get('accuracy'))
        Train_Acc = np.array([Train_Acc]).T
        Valid_Acc = np.asarray(Train_Result_Optimizer.get('val_accuracy'))
        Valid_Acc = np.asarray([Valid_Acc]).T

        # Accumulate the training and validation/test loss after every pooling iteration - for plotting
        Pool_Valid_Loss = np.append(Pool_Valid_Loss, Valid_Loss, axis=1)
        Pool_Train_Loss = np.append(Pool_Train_Loss, Train_Loss, axis=1)
        Pool_Valid_Acc = np.append(Pool_Valid_Acc, Valid_Acc, axis=0)
        Pool_Train_Acc = np.append(Pool_Train_Acc, Train_Acc, axis=0)

        print('Evaluate Model Test Accuracy with pooled points')

        score, acc = model.evaluate(
            X_test, Y_test, verbose=1)
        print('Test score:', score)
        print('Test accuracy:', acc)
        all_accuracy = np.append(all_accuracy, acc)
        # Y_predict_probabilities = model.predict_proba(X_test, batch_size=batch_size)
        # predicted_log_likelihood = log_loss(Y_test, Y_predict_probabilities)
        # rmse = sqrt(mean_squared_error(Y_test, Y_predict_probabilities))

        print('Use this trained model with pooled points for Dropout again')

    print('Storing Accuracy Values over experiments')
    Experiments_All_Accuracy = Experiments_All_Accuracy + all_accuracy

Experiment Number  0
X_train shape: (100, 28, 28, 1)
100 train samples
Distribution of Training Classes: [10 10 10 10 10 10 10 10 10 10]
Training Model Without Acquisitions in Experiment 0
Model: "vgg_like_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            multiple                  320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  18496     
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________

130/130 [==============================] - 1s 8ms/sample - loss: 1.9045 - accuracy: 0.3385 - val_loss: 1.8484 - val_accuracy: 0.4080
Epoch 6/10
130/130 [==============================] - 1s 8ms/sample - loss: 1.8231 - accuracy: 0.4462 - val_loss: 1.7481 - val_accuracy: 0.4878
Epoch 7/10
130/130 [==============================] - 1s 8ms/sample - loss: 1.7374 - accuracy: 0.4692 - val_loss: 1.6691 - val_accuracy: 0.5258
Epoch 8/10
130/130 [==============================] - 1s 8ms/sample - loss: 1.6915 - accuracy: 0.4692 - val_loss: 1.5806 - val_accuracy: 0.5464
Epoch 9/10
130/130 [==============================] - 1s 8ms/sample - loss: 1.5431 - accuracy: 0.5231 - val_loss: 1.4805 - val_accuracy: 0.5546
Epoch 10/10
130/130 [==============================] - 1s 8ms/sample - loss: 1.4916 - accuracy: 0.4846 - val_loss: 1.3867 - val_accuracy: 0.5642
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 2s 236us/sample - loss: 1.3750 - accuracy: 0.5691
T

170/170 [==============================] - 1s 7ms/sample - loss: 1.4464 - accuracy: 0.5294 - val_loss: 1.0941 - val_accuracy: 0.7114
Epoch 7/10
170/170 [==============================] - 1s 7ms/sample - loss: 1.2702 - accuracy: 0.6059 - val_loss: 0.9124 - val_accuracy: 0.7750
Epoch 8/10
170/170 [==============================] - 1s 7ms/sample - loss: 1.1147 - accuracy: 0.6412 - val_loss: 0.8560 - val_accuracy: 0.7692
Epoch 9/10
170/170 [==============================] - 1s 7ms/sample - loss: 0.9695 - accuracy: 0.6412 - val_loss: 0.8095 - val_accuracy: 0.7718
Epoch 10/10
170/170 [==============================] - 1s 7ms/sample - loss: 0.8728 - accuracy: 0.7235 - val_loss: 0.7258 - val_accuracy: 0.8054
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 3s 285us/sample - loss: 0.6968 - accuracy: 0.8126
Test score: 0.6967630520820618
Test accuracy: 0.8126
Use this trained model with pooled points for Dropout again
POOLING ITERATION 7
Dropout Iter

210/210 [==============================] - 1s 5ms/sample - loss: 0.9341 - accuracy: 0.7048 - val_loss: 0.6533 - val_accuracy: 0.8210
Epoch 8/10
210/210 [==============================] - 1s 5ms/sample - loss: 0.8407 - accuracy: 0.7190 - val_loss: 0.5932 - val_accuracy: 0.8346
Epoch 9/10
210/210 [==============================] - 1s 5ms/sample - loss: 0.7535 - accuracy: 0.7095 - val_loss: 0.5691 - val_accuracy: 0.8336
Epoch 10/10
210/210 [==============================] - 1s 5ms/sample - loss: 0.6963 - accuracy: 0.7333 - val_loss: 0.5237 - val_accuracy: 0.8516
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 3s 251us/sample - loss: 0.4968 - accuracy: 0.8624
Test score: 0.4967833930969238
Test accuracy: 0.8624
Use this trained model with pooled points for Dropout again
POOLING ITERATION 11
Dropout Iteration 0
Dropout Iteration 1
Dropout Iteration 2
Dropout Iteration 3
Dropout Iteration 4
Dropout Iteration 5
Dropout Iteration 6
Dropout Iterati

250/250 [==============================] - 1s 4ms/sample - loss: 0.8746 - accuracy: 0.7320 - val_loss: 0.5601 - val_accuracy: 0.8486
Epoch 9/10
250/250 [==============================] - 1s 4ms/sample - loss: 0.7468 - accuracy: 0.7440 - val_loss: 0.5383 - val_accuracy: 0.8476
Epoch 10/10
250/250 [==============================] - 1s 4ms/sample - loss: 0.7523 - accuracy: 0.7320 - val_loss: 0.4967 - val_accuracy: 0.8594
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 2s 249us/sample - loss: 0.4756 - accuracy: 0.8705
Test score: 0.47561847326755524
Test accuracy: 0.8705
Use this trained model with pooled points for Dropout again
POOLING ITERATION 15
Dropout Iteration 0
Dropout Iteration 1
Dropout Iteration 2
Dropout Iteration 3
Dropout Iteration 4
Dropout Iteration 5
Dropout Iteration 6
Dropout Iteration 7
Dropout Iteration 8
Dropout Iteration 9
Acquised Points added to training set
Train on 260 samples, validate on 5000 samples
Epoch 1/10
26

...

640/640 [==============================] - 2s 3ms/sample - loss: 0.7386 - accuracy: 0.7516 - val_loss: 0.3461 - val_accuracy: 0.9070
Epoch 8/10
640/640 [==============================] - 2s 2ms/sample - loss: 0.7048 - accuracy: 0.7437 - val_loss: 0.3015 - val_accuracy: 0.9182
Epoch 9/10
640/640 [==============================] - 1s 2ms/sample - loss: 0.5784 - accuracy: 0.8125 - val_loss: 0.2854 - val_accuracy: 0.9186
Epoch 10/10
640/640 [==============================] - 2s 2ms/sample - loss: 0.5566 - accuracy: 0.8062 - val_loss: 0.2534 - val_accuracy: 0.9262
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 3s 319us/sample - loss: 0.2498 - accuracy: 0.9280
Test score: 0.24977996934056282
Test accuracy: 0.928
Use this trained model with pooled points for Dropout again
POOLING ITERATION 54
Dropout Iteration 0
Dropout Iteration 1
Dropout Iteration 2
Dropout Iteration 3
Dropout Iteration 4
Dropout Iteration 5
Dropout Iteration 6
Dropout Iterati

680/680 [==============================] - 2s 2ms/sample - loss: 0.6608 - accuracy: 0.7809 - val_loss: 0.3136 - val_accuracy: 0.9072
Epoch 9/10
680/680 [==============================] - 2s 3ms/sample - loss: 0.5729 - accuracy: 0.8162 - val_loss: 0.2698 - val_accuracy: 0.9184
Epoch 10/10
680/680 [==============================] - 2s 3ms/sample - loss: 0.4980 - accuracy: 0.8471 - val_loss: 0.2505 - val_accuracy: 0.9270
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 3s 334us/sample - loss: 0.2499 - accuracy: 0.9259
Test score: 0.24985138634443282
Test accuracy: 0.9259
Use this trained model with pooled points for Dropout again
POOLING ITERATION 58
Dropout Iteration 0
Dropout Iteration 1
Dropout Iteration 2
Dropout Iteration 3
Dropout Iteration 4
Dropout Iteration 5
Dropout Iteration 6
Dropout Iteration 7
Dropout Iteration 8
Dropout Iteration 9
Acquised Points added to training set
Train on 690 samples, validate on 5000 samples
Epoch 1/10
69

720/720 [==============================] - 2s 2ms/sample - loss: 0.5760 - accuracy: 0.8083 - val_loss: 0.2542 - val_accuracy: 0.9264
Epoch 10/10
720/720 [==============================] - 2s 2ms/sample - loss: 0.4816 - accuracy: 0.8361 - val_loss: 0.2268 - val_accuracy: 0.9294
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 3s 320us/sample - loss: 0.2178 - accuracy: 0.9354
Test score: 0.21782430109083653
Test accuracy: 0.9354
Use this trained model with pooled points for Dropout again
POOLING ITERATION 62
Dropout Iteration 0
Dropout Iteration 1
Dropout Iteration 2
Dropout Iteration 3
Dropout Iteration 4
Dropout Iteration 5
Dropout Iteration 6
Dropout Iteration 7
Dropout Iteration 8
Dropout Iteration 9
Acquised Points added to training set
Train on 730 samples, validate on 5000 samples
Epoch 1/10
730/730 [==============================] - 3s 4ms/sample - loss: 2.1540 - accuracy: 0.2068 - val_loss: 1.7269 - val_accuracy: 0.3990
Epoch 2/10
73

760/760 [==============================] - 2s 2ms/sample - loss: 0.4152 - accuracy: 0.8658 - val_loss: 0.2081 - val_accuracy: 0.9344
Evaluate Model Test Accuracy with pooled points
10000/10000 [==============================] - 3s 294us/sample - loss: 0.2053 - accuracy: 0.9370
Test score: 0.20530301344394683
Test accuracy: 0.937
Use this trained model with pooled points for Dropout again
POOLING ITERATION 66
Dropout Iteration 0
Dropout Iteration 1
Dropout Iteration 2
Dropout Iteration 3
Dropout Iteration 4
Dropout Iteration 5
Dropout Iteration 6
Dropout Iteration 7
Dropout Iteration 8
Dropout Iteration 9
Acquised Points added to training set
Train on 770 samples, validate on 5000 samples
Epoch 1/10
770/770 [==============================] - 2s 3ms/sample - loss: 2.1828 - accuracy: 0.1922 - val_loss: 1.7763 - val_accuracy: 0.4224
Epoch 2/10
770/770 [==============================] - 2s 2ms/sample - loss: 1.8823 - accuracy: 0.3039 - val_loss: 1.2396 - val_accuracy: 0.6674
Epoch 3/10
770/

 3808/10000 [==========>...................] - ETA: 2s - loss: 0.2382 - accuracy: 0.9236

In [7]:
 Y_train.shape

(100, 10)

In [5]:
print(Train_Result_Optimizer.get('val_accuracy'))

[0.351, 0.409, 0.4398, 0.5694, 0.6906, 0.743, 0.74, 0.734, 0.7234, 0.7386]


In [6]:
Pool_Valid_Acc.shape

(1,)